## Great, now that we discussed a little let's continue

Given that the current approach utilized by the authors lacks reproducibility, we will explore an alternative method by leveraging nf-core pipelines for data analysis.

Please explain, how we will achieve reproducibility for the course  with this approach.


nf-core pipelines provide standardized, versioned pipelines that ensure reproducible results. All input (samplesheets, profiles, additional parameters) can be specified and is reported in logfiles. Therefore a rerun with the same command will result in identical results.

You have successfully downloaded 2 of the fastq files we will use in our study.

What is the next step if we want to first have a count table and check the quality of our fastq files? What is the pipeline called to do so?

`nf-core/rnaseq`

Analyze the 2 files using an nf-core pipeline.

What does this pipeline do?

Which are the main tools that will be used in the pipeline?

rnaseq takes a samplesheet with fastq files, performs QC, trimming, alignment to a reference, and produces a gene expression matrix together with QC reports.

The main tools are:
- FastQC for quality control
- TrimGalore! for adapter trimming
- STAR or HiSAT2 for genome alignment and quantification
- SAMtools, picard, BEDtools for post-processing
- StringTie to assemble RNA-seq alignments into potential transcripts
- RSeQC, DESeq2, MultiQC for final QC

As all other nf-core pipelines, the chosen pipeline takes in a samplesheet as input.

Use Python and pandas to create the samplesheet for your 2 samples. Feel free to make use of the table you created earlier today.

Choose your sample names wisely, they must be the connection of the results to the metadata. If you can't find the sample in the metadata later, the analysis was useless.

In [ ]:
import pandas as pd
# create samplesheet
df = pd.read_csv("./fetchngs/samplesheet/samplesheet.csv")
df = df[["run_accession", "fastq_1", "fastq_2"]]

df['strandedness'] = 'auto'

# replace run_accession by column names in conditions_filtered.csv where there is True
conditions = pd.read_csv('conditions_filtered.csv')

combis = conditions[conditions['Run'].isin(df['run_accession'])]
combis = combis.set_index('Run').apply(lambda x: '-'.join(x.index[x].tolist()), axis=1).reset_index()[0]

# add sample column
df.insert(loc = 0, column = 'sample', value = combis)
df.drop(columns=['run_accession'], inplace=True)
# Write samplesheet.csv
df.to_csv("samplesheet.csv", index=False)

In [ ]:
# Download GRCm39 reference genome and GTF file
!wget https://ftp.ensembl.org/pub/release-115/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.toplevel.fa.gz
!wget https://ftp.ensembl.org/pub/release-115/gtf/mus_musculus/Mus_musculus.GRCm39.115.gtf.gz

--2025-09-30 13:38:47--  https://ftp.ensembl.org/pub/release-115/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.toplevel.fa.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 806418890 (769M) [application/x-gzip]
Saving to: ‘Mus_musculus.GRCm39.dna.toplevel.fa.gz’

Mus_musculus.GRCm39 100%[===================>] 769.06M  5.94MB/s    in 2m 27s  

2025-09-30 13:41:22 (5.22 MB/s) - ‘Mus_musculus.GRCm39.dna.toplevel.fa.gz’ saved [806418890/806418890]

--2025-09-30 13:41:22--  https://ftp.ensembl.org/pub/release-115/gtf/mus_musculus/Mus_musculus.GRCm39.115.gtf.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41681142 (40M) [application/x-gzip]
Saving to: ‘Mus_musculus.GRCm39.115.gtf.gz’

Mus_m

In [50]:
# --genome GRCm39
# post here the command you used to run nf-core/rnaseq
!nextflow run nf-core/rnaseq \
    -r 3.21.0 \
    --input samplesheet.csv \
    --outdir rnaseq_out \
    --gtf Mus_musculus.GRCm39.115.gtf.gz \
    --fasta Mus_musculus.GRCm39.dna.toplevel.fa.gz \
    --save_align_intermeds \
    -profile docker \
    -c nextflow.config \
    -resume


 N E X T F L O W   ~  version 25.04.7

Launching `https://github.com/nf-core/rnaseq` [amazing_neumann] DSL2 - revision: 9738a2df42 [3.21.0]


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/rnaseq 3.21.0
------------------------------------------------------
Input/output options
  input               : samplesheet.csv
  outdir              : rnaseq_out

Reference genome options
  fasta               : Mus_musculus.GRCm39.dna.toplevel.fa.gz
  gtf                 : Mus_musculus.GRCm39.115.gtf.gz

UMI options
  umi_discard_read    : 0

Optional outputs
  save_align_intermeds: true

Generic options
  trace_report_suffix : 2025-10-01_05-46-15

Core Nextflow options
  revision            : 3.21.0
  runName             : amazing_neuma

Explain all the parameters you set and why you set them in this way.



- `-r` version for reproducibility
- `--input` specify samplesheet as input
- `--outdir` directory to write results into
- `-profile` run docker container


- `--fasta` reference genome GRCm39 for Mus musculus
- `--gtf` gene annotations to for the reference

alternatively the `--genome`flag can be set specifying the reference genome GRCm39

- `--save_align_intermeds` save intermediate steps when gererating BAM files for later usage, future runs can use the parameter `--skip_alignment` to reuse BAM files
- `-c` specify config file for memory constraints
- `-resume` from previous runs

## Browsing the results

How did the pipeline perform?

Explain the quality control steps. Are you happy with the quality and why. If not, why not.
Please give additional information on : 
- ribosomal rRNA
- Duplication
- GC content

What are the possible steps that could lead to poorer results?

Would you exclude any samples? If yes, which and why?

What would you now do to continue the experiment? What are the scientists trying to figure out? Which packages on R or python would you use?